### Git分支
几乎每一种版本控制系统都以某种形式支持分支。使用分支意味着你可以从开发主线上分离出来，然后在不影响主线同时继续工作。
Git分支是难以置信的轻量级的，它的新建操作可以瞬间完成。Git鼓励在工作流程中频繁使用分支和合并。

#### <font color=#8D0000>何为分支</font>
Git是如何储存数据的：Git保存的不是文件的差异，而只是一系列文件快照。


在Git提交时，都会保存一个提交（commit）对象，该对象包含一个指向暂存内容快照的指针，包含本次提交的作者等相关附属信息，包含零个或多个指向该提交对象的父指针。

暂存操作会对每一个文件计算校验和，然后把当前版本的文件快照保存到Git仓库，并将校验和加入暂存区域
当使用`git commit`新建一个提交对象前，Git会先计算每一个子目录的校验和，然后在Git仓库保存为树（tree）对象。之后Git创建提交对象，除了包含相关提交对象信息外，还包含指向这个树对象的指针。

Git仓库中的对象有：<font color=#FFD700>表示文件快照内容的blob对象；记录着目录树内容及其中各个文件对应blob对象索引的tree对象；包含指向tree对象的索引和其他提交信息元数据的commit对象。</font>

现在谈谈分支。Git中分支本质上是一个指向commit对象的可变指针。Git使用master作为分支的默认名字。它在每次提交的时候都会自动向前移动。

创建新的分支指针，比如新建一个testing分支，可以使用以下命令：
```
$ git branch testing
```

这会在当前的commit对象上新建一个分支指针。

Git保存着一个名为HEAD的特殊指针。它是一个指向你正在工作中的本地分支的指针。运行`git branch`命令，仅仅新建了一个新的分支，但不会自动切换到这个分支。

要切换到其他分支，可以执行`git checkout`命令。转换到testing分支：
```
$ git checkout testing
```
这样，HEAD就指向了testing分支。

```
$ vim test.rb
$ git commit -a -m "made a change"
$ git checkout master
```

它把HEAD指针移回了master分支，并把工作目录换成了master分支所指向快照的内容。也就是说现在所做的改动始于一个较老的版本。它主要的作用是将testing分支作出的修改暂时取消，这样你就可以向另一个方向进行开发了。





#### <font color=#8D0000>分支的新建与切换</font>
现在让我们来看一个简单的分支与合并的例子，实际工作中大体也会用到这样的工作流程：
1. 开发某个网站
2. 为实现某个新的需求，新建一个分支
3. 在这个分支开展工作

假设此时，收到一个很严重的问题去处理，可以按照下面的方式处理：
1. 返回原先已经发布到生产服务器的分支
2. 为这次紧急修补建立一个新的分支，并在其中修复BUG
3. 通过测试后，，切换到生产服务器所在的分支，将修补分支合并进来，然后推送到生产服务器
4. 切换到之前实现新需求的分支，继续工作

##### <font color=#00CED1>分支的新建与切换</font>
新建并切换到该分支，运行一下命令：
```
$ git checkout -b hotfix
```

这相当于下面两条命令：
```
$ git branch hotfix
$ git checkout hotfix
```

<font color=#FFD700>不过在切换回master分支前，留心你的暂存区和工作区里，那些还没有提交的修改，它会和你即将检出的分支产生冲突从而阻止Git为你切换分支。切换分支时最好保持一个清洁的工作区域。</font>稍后会介绍几个绕过这种问题的方法（分别叫stashing和commit amending）。
```
$ git checkout master
```

此时工作目录中的内容和你解决问题之前一样。有一点要牢记：Git会把工作目录的内容恢复为检出某一分支时它所指向的那个提交对象的快照。它会自动添加、删除和修改文件以确保目录内容和你当前提交时完全一样。

有必要做一些测试，确保修补是成功的，<font color=#FFD700>然后回到master分支并把它合并进来</font>，然后发布到生产服务器。用 git merge 命令进行合并：
```
$ git checkout master
$ git merge hotfix
```

请注意，合并时出现了“Fast forward”的提示。由于当前master分支所在的提交对象是要并入的 hotfix 分支的直接上游，Git 只需把master分支指针直接右移。因为这种单线的历史分支不存在任何需要解决的分歧，所以这种合并过程可以称为快进（Fast forward）。

由于当前的hotfix分支和master分支都指向相同的提交对象，所以hotfix已经完成了历史使命，可以删掉了。使用Git branch的-d选项执行删除操作：
```
$ git branch -d hotfix
```

##### <font color=#00CED1>分支的合并</font>
Git 用两个分支的末端（$C_4$ 和 $C_5$）以及它们的共同祖先（$C_2$）进行一次简单的三方合并计算。下图红框标出了Git用于合并的三个提交对象：
    
![分支合并][merge]

[merge]: https://github.com/ming-lei-lee/Git_learning/blob/master/picture/tripal_merge.png "merge branch"

这次，Git并没有简单的把分支指针右移，而是对三方合并后的结果重新做一个新的快照，并自动创建一个指向它的提交对象（$C_6$）。这个提交对象比较特殊，它有两个祖先（$C_4$和$C_5$）



##### <font color=#00CED1>遇到冲突时的分支合并</font>
有时候合并操作并不会如此顺利。如果在不同的分支中都修改了同一个文件的同一部分，Git就无法干净地把两者合到一起（这种问题只能由人来裁决）

Git 作了合并，但没有提交，它会停下来等你解决冲突，可以用git status查阅

任何包含未解决冲突的文件都会以未合并（unmerged）的状态列出。Git会在有冲突的文件里加入标准的冲突解决标记，可以通过它们来手工定位并解决这些冲突。

![]()

<font color=#FFD700>可以看到`======`隔开的上半部分，是`HEAD`（即`master`分支）中的内容，下半部分是在`iss53`分支中的内容。解决冲突的办法无非是二者选其一或者由你来亲自整合到一起。</font>

在解决所有文件里的冲突后，运行`git add`将把它标记为已解决状态（实际上就是来一次快照保存到暂存区域）。因为一旦暂存，就表示冲突已经解决。

再运行一次`git status`来确认所有的冲突都已经解决。如果觉得满意了，并且确认所有冲突都已解决，也就进入了暂存区，用`git commit`来完成这次合并提交。






#### <font color=#8D0000>分支的管理</font>










### <font color=#8D0000>远程分支</font>
远程分支（remote branch）是对远程仓库中分支的索引。它们是<font color=#FFD700>一些无法移动的本地分支；只有Git进行网络交互时才会更新</font>，也就是说只要你不和服务器通讯，你的 origin/master 指针依然保持原位不会移动。远程分支就像是书签，提醒你上次连接远程仓库时上面各分支的位置。

我们用（远程仓库名）/（分支名）这样的分支形式表示远程分支。

可以运行 git fetch origin 来同步远程服务器上的数据到本地。该命令首先找到 origin 是哪个服务器，从上面获取你尚未拥有的数据，更新你本地的数据库，然后把 origin/master 的指针移动到最新的位置上。

为了演示拥有多个远程分支（在不同远程服务器上）的项目是如何工作的。假设还有另一个供敏捷开发小组使用的内部服务器 git.team1.ourcompany.com 。可以用 git remote add 命令把它加为当前项目的远程分支之一。
```
$ git remote add  replace_name URL
$ git fetch replace_name
```
由于当前该服务器上的内容是你 origin 服务器上的子集，Git不会下载任何数据，而是简单创建一个名为 relace_name 的远程分支。

##### <font color=#00CED1>推送本地分支</font>
创建的本地分支不会因为写入操作而自动的同步到引用的远程分支，需要明确执行推送分支操作。

如果你有一个分支需要他人一起开发，可以运行 git push (远程仓库)（分支名） 。也可以运行 git push origin (本地分支：远程分支) 来实现相同的功能。通过此语法可以将本体分支推送到某个命名不同的远程分支。

值得注意的是，在 fetch 操作下载好新的运程分支后，你仍然无法本地编辑该远程仓库中的分支。换句话说，你不会有一个新的 serverfix 分支，只是一个无法移动的 origin/serverfix 指针。

如果要把远程分支的内容合并到当前分支， 可以运行 git merge origin/serverfix 。如果想要一份自己的 serverfix 来开发，可以在远程分支的基础上分化一个新的分支：
```
$ git branch -b serverfix origin/serverfix
```
这会切换到新建的 serverfix 本地分支，其内容同远程分支 origin/serverfix 一致。




